In [1]:
# Add a command to produce plots directly below cells.
%matplotlib inline
import numpy as np
import os
# Prints current working directory.
#os.getcwd()
# Change working directory.
os.chdir("/scratch/w40/gp8431/run")
# List files in the current directory.
#os.listdir()

In [2]:
# Import xarray and glob for data analysis.
import xarray as xr
from glob import glob

# Dask for processing
#import dask.array as da
import dask as da

In [3]:
# This cell will extract era data and srfamp = 10.0 model values, then 
# compute the anomaly for each, and the difference in anomaly.

# Extract model output (no heating and heating = 10.0)
os.chdir("/scratch/w40/gp8431/run/r0_1101_srfamp_10.0")
ih_for = xr.open_dataset("plevel_daily_ih.nc", decode_times = False)
ih_for_davg = ih_for.mean('time')

os.chdir("/scratch/w40/gp8431/run/r0_1101_srfamp_0.0")
nh_for = xr.open_dataset("plevel_daily_ih.nc", decode_times = False)
nh_for_davg = nh_for.mean('time')

In [4]:
# Compute anomaly based on extracted model values.
mod_anom = ih_for_davg-nh_for_davg

In [5]:
# Extract era data

files = glob('/g/data/rt52/era5/pressure-levels/monthly-averaged/t/*/*')
files.sort()
# Add combine option based on error message.

# COULD SPECIFY CHUNKING HERE!

era_all = xr.open_mfdataset(files, combine='by_coords')
era_clim = era_all.groupby('time.month').mean()

In [6]:
# Attempt to interpolate era_all onto coarser grid and change coordinate name to match model.
#era_all = era_all.rename({"latitude":"lat", "longitude":"lon", "level":"pfull"})
#era_all['lon'] = (era_all['lon'] + 180.0)
#erai = era.interp_like(ihd_avg)

In [7]:
# Plotting issue?
#era_clim.sel(month=8,level=850).t.plot()
era_anom = era_all.sel(time=slice('2019-01-01','2019-12-31')).groupby('time.month')-era_clim

In [8]:
# Attempt to interpolate era_anom onto coarser grid and change coordinate names to match model anomalies for comparison.
era_anom = era_anom.rename({"latitude":"lat", "longitude":"lon", "level":"pfull", "t":"temp"})
era_anom['lon'] = (era_anom['lon'] + 180.0)
era_anom_interp = era_anom.interp_like(mod_anom)

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-20.07/lib/python3.7/site-packages/xarray/core/indexing.py:1361: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


In [9]:
# Select the era_anom data from August for comparison with perpet_day = 220 in the model.
era_anom_interp_aug = era_anom_interp.sel(time=slice('2019-08-01', '2019-08-31')).squeeze()
#print(era_anom_interp_aug)
era_anom_interp_all = era_anom_interp.mean('time')
#print(era_anom_interp_all)

In [10]:
# Compute the difference in temp anomalies between the model and the data
# at 925hPa.
# Here I use the average era anomaly taken over 500 days.

#era_anom_interp_all.sel(pfull=925.0).plot()

# REMOVE .TEMP 
era_temp_anom = era_anom_interp_all.sel(pfull=925.0, method='nearest')
mod_temp_anom = mod_anom.sel(pfull=925.0, method='nearest')


In [11]:
# Convert both model and data temperature anomalies to arrays.
#a = xr.Dataset.to_array(era_temp_anom)

#print(era_temp_anom)
#print(mod_temp_anom)

#diff = era_temp_anom - mod_temp_anom
#print(diff)

#a = xr.DataArray.values(era_temp_anom)
#print(a)

# The variables era_temp_anom and mod_temp_anom are both DataArray type
# data structures - they contain just a single data variable (temp).
# The coordinates denote where the temperature value is measured.
# The dask array storage method cuts a large array into smaller arrays,
# so computations can be made on arrays larger than memory!
# In this case, it seems there is one numpy array for each coordinate 
# and therefore one for each temp value?
# era_temp_anom is a dask array, whereas mod_temp_anom is simply an array
# I want to convert both to the same type in order to compute and plot
# the difference.

# Convert dask array to numpy array
#era_temp_anom = np.asarray(era_temp_anom)

#print(era_temp_anom)
#print(mod_temp_anom)

# Convert numpy array to dask array
#mod_temp_anom = da.from_array(mod_temp_anom, chunks=(64, 128))

print("ERA")
print(era_temp_anom)
#era_temp_anom
print("MOD")

# PRINT THIS! 
print(mod_temp_anom)
#mod_temp_anom.temp[0][0].values

# CHANGE DIFF PARAMETERS TO SEE WHICH ARGUMENT HAS 'UNHASHABLE TYPE'
#diff = xr.DataArray.diff(mod_temp_anom, mod_temp_anom)
diff = era_temp_anom-mod_temp_anom
print("DIFF")
#diff
#diff.temp
diff
#diff
# Attempt to compare temperature anomalies for era and model.
#diff.temp.values()

ERA
<xarray.Dataset>
Dimensions:  (lat: 64, lon: 128)
Coordinates:
  * lon      (lon) float64 0.0 2.812 5.625 8.438 ... 348.8 351.6 354.4 357.2
  * lat      (lat) float64 -87.86 -85.1 -82.31 -79.53 ... 79.53 82.31 85.1 87.86
    pfull    float64 925.0
Data variables:
    temp     (lat, lon) float32 dask.array<chunksize=(64, 128), meta=np.ndarray>
MOD
<xarray.Dataset>
Dimensions:  (lat: 64, latb: 65, lon: 128, lonb: 129)
Coordinates:
  * lon      (lon) float64 0.0 2.812 5.625 8.438 ... 348.8 351.6 354.4 357.2
  * lonb     (lonb) float64 -1.406 1.406 4.219 7.031 ... 350.2 353.0 355.8 358.6
  * lat      (lat) float64 -87.86 -85.1 -82.31 -79.53 ... 79.53 82.31 85.1 87.86
  * latb     (latb) float64 -90.0 -86.58 -83.76 -80.96 ... 83.76 86.58 90.0
    pfull    float32 925.0
Data variables:
    hght     (lat, lon) float32 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    ps       (lat, lon) float32 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    ucomp    (lat, lon) float32 0.0 0.0 0.0 0

<xarray.Dataset>
Dimensions:  (lat: 64, latb: 65, lon: 128, lonb: 129)
Coordinates:
  * lon      (lon) float64 0.0 2.812 5.625 8.438 ... 348.8 351.6 354.4 357.2
  * lat      (lat) float64 -87.86 -85.1 -82.31 -79.53 ... 79.53 82.31 85.1 87.86
    pfull    float64 925.0
  * lonb     (lonb) float64 -1.406 1.406 4.219 7.031 ... 350.2 353.0 355.8 358.6
  * latb     (latb) float64 -90.0 -86.58 -83.76 -80.96 ... 83.76 86.58 90.0
Data variables:
    temp     (lat, lon) float32 dask.array<chunksize=(64, 128), meta=np.ndarray>

In [13]:
# Try extracting a slice and looking at its values

#diff.temp.values

# TRY TO CALCULATE THE DIFFERENCE USING DIFF!
#diff = era_temp_anom.diff({'temp':("x", mod_temp_anom)})

In [14]:
diff.temp.shape

(64, 128)

In [15]:
diff.temp

<xarray.DataArray 'temp' (lat: 64, lon: 128)>
dask.array<sub, shape=(64, 128), dtype=float32, chunksize=(64, 128), chunktype=numpy.ndarray>
Coordinates:
  * lon      (lon) float64 0.0 2.812 5.625 8.438 ... 348.8 351.6 354.4 357.2
  * lat      (lat) float64 -87.86 -85.1 -82.31 -79.53 ... 79.53 82.31 85.1 87.86
    pfull    float64 925.0

In [16]:
avg = diff.temp.mean(dim='lon').mean(dim='lat')

In [17]:
#avg.values

In [23]:
diff_IOD = diff.temp.sel(lat=0.0, lon=95.0, method='nearest')
# This line is very slow!
# diff_IOD.values

In [22]:
diff_IOD

<xarray.DataArray 'temp' ()>
dask.array<getitem, shape=(), dtype=float32, chunksize=(), chunktype=numpy.ndarray>
Coordinates:
    lon      float64 95.62
    lat      float64 1.395
    pfull    float64 925.0